<a href="https://colab.research.google.com/github/AjaybirRandhawa/Face-Generator/blob/main/Face_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive/')
import os

os.getcwd()

TensorFlow 1.x selected.
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content'

In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 138 (delta 4), reused 10 (delta 4), pack-reused 127
Receiving objects: 100% (138/138), 599.47 KiB | 3.11 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2")
import dnnlib

In [ ]:
import argparse
import numpy as np
import PIL.Image
import dnnlib.tflib as tflib
import re

import pretrained_networks

#Takes short seed number, expands it
def expand_seed(seeds, vector_size):
  results = []

  #Takes the seed states and makes it into a number of the length then vectorsize from that seed state, this way we can duplicate the randomness
  for seed in seeds:
    rnd = np.random.RandomState(seed)
    results.append(rnd.randn(1, vector_size))
    
  return results

#Based on code from the nvidia git library, GS -> NN, seeds = seed vectors, truncation_psi is what stylegan2 uses to make images better res 
def generate_images(GS, seeds, truncation_psi):
  #Noise will account for wrinkles and hair movement etc on seeds generated
  noise_vars = [var for name, var in GS.components.synthesis.vars.items() if name.startswith('noise')]

  #Converts 
  GS_kwargs = dnnlib.EasyDict()
  GS_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
  GS_kwargs.randomize_noise = False

  if truncation_psi is not None:
    GS_kwargs.truncation_psy = truncation_psi

  for seed_index, seed in enumerate(seeds):
    print('Generating image for seed %d/%d' % (seed_index, len(seeds)))
    rnd = np.random.RandomState() #fix it so while it stays the same face, it will introduce small variations like hair placement
    tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) #height and width of images
    #makes images and stores them
    images = GS.run(seed, None, **GS_kwargs) #Minibatch, height, width, channel
    path = f"/content/drive/My Drive/projects/tmp/image{seed_index}.png"
    PIL.Image.fromarray(images[0], 'RGB').save(path)

In [ ]:
#Load the neural network
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects"
sc.run_desc = 'generate-images'
network_pki = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'

print('Loading networks from %s' % network_pki)
_G, _D, GS = pretrained_networks.load_networks(network_pki)
vector_size = GS.input_shape[1:][0]
seeds = expand_seed(range(8000,8020), vector_size)
generate_images(GS, seeds, truncation_psi=0.5)

Loading networks from gdrive:networks/stylegan2-ffhq-config-f.pkl
Generating image for seed 0/20
Generating image for seed 1/20
Generating image for seed 2/20
Generating image for seed 3/20
Generating image for seed 4/20
Generating image for seed 5/20
Generating image for seed 6/20
Generating image for seed 7/20
Generating image for seed 8/20
Generating image for seed 9/20
Generating image for seed 10/20
Generating image for seed 11/20
Generating image for seed 12/20
Generating image for seed 13/20
Generating image for seed 14/20
Generating image for seed 15/20
Generating image for seed 16/20
Generating image for seed 17/20
Generating image for seed 18/20
Generating image for seed 19/20


In [ ]:
#Load the neural network
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'

print("Loading networks from the %s" % network_pkl)
_G, _D, GS = pretrained_networks.load_networks(network_pkl)
vector_size = GS.input_shape[1:][0]
seeds = expand_seed([4256, 4269], vector_size)

Loading networks from the gdrive:networks/stylegan2-ffhq-config-f.pkl


In [ ]:
steps = 300
diff = seeds[1] - seeds[0]
step = diff/steps
current = seeds[0].copy()

seeds2 = []
for i in range(steps):
  seeds2.append(current)
  current = current + step


generate_images(GS, seeds2, truncation_psi=1.0)

Generating image for seed 0/300
Generating image for seed 1/300
Generating image for seed 2/300
Generating image for seed 3/300
Generating image for seed 4/300
Generating image for seed 5/300
Generating image for seed 6/300
Generating image for seed 7/300
Generating image for seed 8/300
Generating image for seed 9/300
Generating image for seed 10/300
Generating image for seed 11/300
Generating image for seed 12/300
Generating image for seed 13/300
Generating image for seed 14/300
Generating image for seed 15/300
Generating image for seed 16/300
Generating image for seed 17/300
Generating image for seed 18/300
Generating image for seed 19/300
Generating image for seed 20/300
Generating image for seed 21/300
Generating image for seed 22/300
Generating image for seed 23/300
Generating image for seed 24/300
Generating image for seed 25/300
Generating image for seed 26/300
Generating image for seed 27/300
Generating image for seed 28/300
Generating image for seed 29/300
Generating image for

In [ ]:
! ffmpeg -r 30 -i /content/drive/My\ Drive/projects/tmp/image%d.png -vcodec mpeg4 -y /content/drive/My\ Drive/projects/movie.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
! file.download(/content/drive/My\ Drive/projects/movie.mp4)

/bin/bash: -c: line 0: syntax error near unexpected token `/content/drive/My\ Drive/projects/movie.mp4'
/bin/bash: -c: line 0: ` file.download(/content/drive/My\ Drive/projects/movie.mp4)'
